In [ ]:
## 字典方式读取，但是大文件读取太慢了
# from ruamel import yaml

# yaml_path = r'H:\WenLabData\TestYamlData\test.yaml'

# # 使用ruamel.yaml并跳过YAML文件的第一行
# with open(yaml_path) as fin:
#     fin.readline()
#     rfr = yaml.round_trip_load(fin)
# # print(rfr)
# a = rfr['Frames']
# print(rfr['Frames'])

In [ ]:
# # 速度也是太慢了
# import yaml
# yaml_path = r'H:\WenLabData\TestYamlData\del_yaml.yaml'
# f = open(yaml_path, 'r', encoding='utf-8')
# cfg = f.read()
# print(type(cfg))  # 读出来是字符串
# print(cfg)

# d = yaml.sefe_load(cfg)  # 用load方法转字典
# f.close()
# # print(d)
# print(type(d))

In [ ]:
import os

# dataset_path = 'Z:\\data2\\WormPoseData\\ProcessingData\\Temp\\20210516_2051_w1'

# print()
# wormname = dataset_path.split('\\')[-1]
# print(wormname)
# yaml_path = os.path.join(dataset_path,wormname,wormname+'.yaml')


# yaml_path = r'H:\WenLabData\TestYamlData\test.yaml'

# yaml_path = r'H:\WenLabData\TestYamlData\20210516_2051_w1.yaml'

# f = open(yaml_path, 'r', encoding='utf-8')
# cfg = f.read()
# print(type(cfg))  # 读出来是字符串
# print(cfg)

# d = yaml.load(cfg)  # 用load方法转字典
# print(d)
# print(type(d))

# from ruamel import yaml



In [57]:
import os
import numpy as np
workpath = os.getcwd()

yaml_path = os.path.join(workpath,'YamlFiles','20210516_2051_w1.yaml')
Yaml_file = []


with open(yaml_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.rstrip('\n') # 删除末尾的回车
        line = line.strip() # 删除前面的空格
        Yaml_file.append(line)
    # lines = f.readlines()



def Get_Frames_line(lines):
    # 获取'Frames: ' 行数
    import re
    for i in range(30):
        if re.match('Frames',lines[i]):
            break
    return(i)

def Get_First_Frames(lines):
    linenumber = Get_Frames_line(lines)+2
    FramsNumber = lines[linenumber].split(':')[-1]
    return(int(FramsNumber))

def Get_End_Frames(lines):
    # 获取最后一个Frames的开始一行
    linenumber = lines[-98].split(':')[-1]
    return(int(linenumber))

def Get_One_Frame(lines,line_start):
    # 获得一帧的所有数据进入一个list
    # lines是读取的yaml文件,是一个list
    # line_start是这一帧开始的行
    OneFrameList = []
    for i in range(line_start,line_start+98):
        OneFrameList.append(lines[i])
    return(OneFrameList)

def Get_Any_Frame(lines,number):
    # 获取任意指定帧的数据
    # 从第1帧开始数
    begin_num = Get_First_Frames(lines)
    end_num = Get_End_Frames(lines)
    All_Frames_num = end_num-begin_num+1  # 总的帧数
    if number < 1 or number>All_Frames_num:
        print("please input:[",1,",",All_Frames_num,"]")
        return(0)
    else:
        frames_line_num = Get_Frames_line(lines)+2+(number-1)*99
        return(Get_One_Frame(lines,frames_line_num))

def Get_ExperimentTime(lines:list):
    # 获取'ExperimentTime: '
    import re
    for i in range(20):
        if re.match('ExperimentTime',lines[i]):
            break
    str0 = lines[i].split('ExperimentTime')
    str0 = str0[1].strip(':') # 去除冒号
    str0 = str0.replace('"',"") # 去除引号 ""
    str0 = str0.replace('\\n',"") # 删除末尾的回车
    ExperimentTime = str0
    return(ExperimentTime)



class YamlFrames(object):
    def __init__(self,name: str, framessize: int):
        self.name = name
        self.ExperimentTime = 'Sun May 16 20:51:33 2021'
        self.DefaultGridSizeForNonProtocolIllum = np.array([20,100])
        self.DefaultGridSizeForNonProtocolIllum = np.zeros((framessize,2)) # DefaultGridSizeForNonProtocolIllum
        self.FrameNumber = np.zeros((framessize,1))  #internal frame number, not nth recorded frame
        self.TimeElapsed = np.zeros((framessize,1)) #time since start of experiment (in s) = sElapsed+ 0.001*msRemElapsed
        self.BoundaryA = np.zeros((framessize,2,100)) # N*2*100 x,y BoundaryA position in pixels on camera
        self.BoundaryB = np.zeros((framessize,2,100)) # N*2*100 x,y BoundaryB position in pixels on camera
        self.SegmentedCenterline = np.zeros((framessize,2,100))  # N*2*100 x,y centerline position in pixels on camera
        self.Head = np.zeros((framessize,2)) #position in pixels on camera
        self.Tail = np.zeros((framessize,2)) #position in pixels on camera
        self.DLPisOn = np.zeros((framessize,1)) #bool whether DLP is active
        self.FloodLightIsOn = np.zeros((framessize,1)) #flood light overrides all other patterns and hits entire fov
        self.IllumInvert = np.zeros((framessize,1)) #whether pattern is inverted (invert has precedence over floodlight)
        self.IllumFlipLR = np.zeros((framessize,1)) #flips output left/right with respect to worm's body
        self.IllumRectOrigin = np.zeros((framessize,2)) #center of the freehand rectangular illumination in wormspace
        self.IllumRectRadius = np.zeros((framessize,2)) #xy value describing dimension of rectangle
        self.StageVelocity = np.zeros((framessize,2)) #velocity sent to stage in stage units/second
        self.ProtocolIsOn = np.zeros((framessize,1)) #bool whether you're using protocol
        self.ProtocolStep = np.zeros((framessize,1)) #what step within protocol is currently selected
        self.GreenLaser = np.zeros((framessize,1))  #int 0-100 of relative laser power. -1 means leaser is not being controlled programmatically
        self.BlueLaser = np.zeros((framessize,1)) #int 0-100 of relative laser power. -1 means leaser is not being controlled programmatically
        self.HeadCurv = np.zeros((framessize,1)) #curvature of the head
        self.HeadCurvDeriv = np.zeros((framessize,1)) #derivative of curvature of the head
        self.StagePosition = np.zeros((framessize,2))
        self.StageFeedbackTarget = np.zeros((framessize,2))

    # def Get_ExperimentTime(self,lines):
    #     # 输入List
        




In [121]:
# def Data_Dim1_To_2D(data):
#     import numpy as np
#     # 一维numpy数据 x1,y1,x2,y2....,xn,yn变为二维
#     # x: x1,x2,...
#     # y: y1,y2,...
#     result = np.zeros((2,100))
#     for i in data:
#         result[i,i+1]
#     data.shape


def ConvertData(lists:list,begin_data:int,end_data:int):
    import numpy as np

    # 将begin_data~end_data 连起来变为 2*100的数组
    data0 = []
    for i in range(begin_data,end_data+1):
        data0.append(lists[i])
    data0 = ''.join(data0)
    data = data0[8:-2] # 去除前面的和后面的字符
    data = list(map(int,data.split(',')))  # 将list中的字符串转为int
    data = np.array(data)
    data0 = np.zeros((2,100))
    data0[0,:] = data[0:200:2]  # x隔一个取一个值
    data0[1,:] = data[1:200:2]  # y隔一个取一个值
    return(data0)


class Extract_OneFrame(object):
    # 
    def __init__(self,OneFrameList:list):
        import numpy as np

        self.FrameNumber = OneFrameList[0].split(':')[-1]
        self.TimeElapsed = int(OneFrameList[1].split(':')[-1])+0.001*float(OneFrameList[2].split(':')[-1])  # 时间
        self.Head = np.array([int(OneFrameList[4].split(':')[-1]),int(OneFrameList[5].split(':')[-1])])
        self.Tail = np.array([int(OneFrameList[7].split(':')[-1]),int(OneFrameList[8].split(':')[-1])])

        self.BoundaryA = ConvertData(OneFrameList,13,29)

        self.BoundaryB = ConvertData(OneFrameList,34,50)

        self.Centerline = ConvertData(OneFrameList,55,71)

        self.DLPisOn =  int(OneFrameList[72].split(':')[-1])
        self.FloodLightIsOn = int(OneFrameList[73].split(':')[-1])
        self.IllumInvert = int(OneFrameList[74].split(':')[-1])
        self.IllumFlipLR = int(OneFrameList[75].split(':')[-1])
        self.IllumRectOrigin = np.array([int(OneFrameList[77].split(':')[-1]),int(OneFrameList[78].split(':')[-1])])
        self.IllumRectRadius = np.array([int(OneFrameList[80].split(':')[-1]),int(OneFrameList[81].split(':')[-1])])
        self.StageVelocity = np.array([int(OneFrameList[83].split(':')[-1]),int(OneFrameList[84].split(':')[-1])])
        self.StagePosition = np.array([int(OneFrameList[86].split(':')[-1]),int(OneFrameList[87].split(':')[-1])])
        self.StageFeedbackTarget = np.array([int(OneFrameList[91].split(':')[-1]),int(OneFrameList[92].split(':')[-1])])
        self.FirstLaser = int(OneFrameList[94].split(':')[-1])
        self.SecondLaser = int(OneFrameList[95].split(':')[-1])
        self.ProtocolIsOn = int(OneFrameList[96].split(':')[-1])
        self.ProtocolStep = int(OneFrameList[97].split(':')[-1])
        self.LaserPower = OneFrameList[93].split(':')[-1]
        # self.GreenLaser = 
        # self.BlueLaser =
        # self.HeadCurv = 
        # self.HeadCurvDeriv =



b = Get_Any_Frame(Yaml_file,2)

OneF = Extract_OneFrame(b)
a = OneF.BoundaryA
print(OneF.BoundaryA)
print(OneF.BoundaryB)
# print(a.shape)

[[608. 609. 609. 608. 605. 608. 607. 603. 598. 600. 598. 596. 588. 580.
  582. 580. 576. 574. 566. 559. 558. 564. 562. 553. 547. 539. 531. 526.
  523. 520. 515. 510. 510. 510. 509. 506. 504. 500. 498. 496. 494. 492.
  494. 493. 491. 487. 480. 482. 482. 481. 479. 479. 481. 478. 476. 474.
  472. 471. 470. 469. 468. 467. 465. 465. 463. 458. 455. 456. 454. 449.
  447. 446. 447. 446. 444. 441. 442. 437. 434. 431. 429. 432. 430. 427.
  424. 421. 421. 418. 415. 414. 411. 408. 409. 406. 403. 400. 397. 395.
  392. 388.]
 [365. 359. 351. 343. 335. 344. 339. 331. 323. 326. 323. 321. 315. 312.
  312. 312. 310. 310. 308. 307. 308. 308. 307. 308. 309. 311. 316. 320.
  323. 327. 334. 340. 343. 342. 345. 351. 353. 358. 359. 361. 363. 365.
  363. 364. 365. 369. 377. 375. 374. 376. 379. 378. 376. 380. 383. 387.
  390. 392. 396. 398. 401. 404. 408. 408. 411. 414. 416. 415. 416. 418.
  419. 420. 419. 420. 420. 421. 421. 422. 422. 422. 422. 422. 422. 422.
  422. 421. 421. 421. 420. 420. 419. 418. 418. 417.

In [123]:
print(OneF.Centerline)
print(OneF.Head)

[[607. 605. 603. 600. 598. 596. 593. 591. 588. 586. 583. 581. 578. 576.
  573. 571. 568. 565. 563. 560. 558. 555. 553. 550. 547. 545. 542. 540.
  537. 535. 533. 531. 529. 527. 525. 523. 521. 519. 518. 516. 515. 513.
  512. 509. 507. 505. 504. 503. 501. 499. 497. 495. 493. 491. 488. 486.
  484. 482. 480. 478. 476. 474. 472. 470. 467. 465. 464. 462. 460. 458.
  456. 454. 452. 449. 447. 444. 442. 439. 437. 434. 432. 429. 427. 424.
  422. 419. 416. 414. 411. 409. 407. 404. 402. 400. 398. 396. 393. 391.
  389. 387.]
 [368. 367. 366. 365. 365. 364. 362. 361. 361. 360. 359. 358. 357. 357.
  357. 356. 356. 355. 355. 355. 355. 355. 354. 354. 354. 354. 355. 356.
  357. 358. 359. 361. 363. 365. 366. 368. 370. 372. 374. 376. 378. 380.
  382. 384. 386. 387. 389. 391. 393. 395. 397. 399. 400. 401. 403. 404.
  406. 407. 409. 410. 412. 413. 415. 417. 418. 420. 422. 424. 425. 427.
  429. 431. 432. 433. 433. 434. 434. 434. 435. 435. 436. 436. 435. 435.
  434. 434. 433. 432. 431. 430. 429. 428. 427. 425.

In [62]:

begin_num1 = Get_First_Frames(Yaml_file)
end_num = Get_End_Frames(Yaml_file)
All_Frames_num = end_num-begin_num1+1  # 总的帧数

b = Get_Any_Frame(Yaml_file,2)
c = Get_Any_Frame(Yaml_file,3)
d = Get_Any_Frame(Yaml_file,43228)


wormname = "20210516_2051_w1"

# 初始化类
YamlFiles = YamlFrames(wormname,All_Frames_num)
YamlFiles.ExperimentTime = Get_ExperimentTime(Yaml_file)
print(YamlFiles.ExperimentTime)

 Sun May 16 20:51:33 2021


In [52]:
class Circle(object):  # 创建Circle类
   def __init__(self, R):  # 约定成俗这里应该使用r，它与self.r中的r同名
       self.r = R

circle1 = Circle(1)  

In [8]:
print(Get_First_Frames(Yaml_file))
print(Get_End_Frames(Yaml_file))

a = Get_Frames_line(Yaml_file)
print(Yaml_file[a+2])


8714
51941
FrameNumber: 8714
